# Fullstack Data Application: Sales Dashboard

## Building a Web-Based Analytics Dashboard with Django + SQL + Pandas + Matplotlib

This capstone project demonstrates how to build a complete fullstack data application that combines:
- **Django** - Web framework for handling HTTP requests and rendering templates
- **SQL/ORM** - Database operations using Django's ORM
- **Pandas** - Data manipulation and analysis
- **Matplotlib** - Chart generation for data visualization

---

## 1. Project Overview

### What We're Building

A web-based Sales Dashboard that displays:
- Total revenue and key metrics
- Sales breakdown by region (pie chart)
- Monthly sales trends (line chart)
- Top-selling products (bar chart)

### Architecture Diagram

```
+-------------------+     +------------------+     +-------------------+
|                   |     |                  |     |                   |
|   Browser/Client  |<--->|  Django Views    |<--->|  SQLite Database  |
|                   |     |                  |     |                   |
+-------------------+     +--------+---------+     +-------------------+
                                   |
                                   v
                    +-----------------------------+
                    |                             |
                    |   Analytics Service         |
                    |   (Pandas + Matplotlib)     |
                    |                             |
                    +-----------------------------+
```

### Data Flow

1. User requests the dashboard page
2. Django view queries the database via ORM
3. Analytics service processes data with Pandas
4. Matplotlib generates charts as base64 images
5. Template renders the complete dashboard

### Prerequisites

```bash
pip install django pandas matplotlib
```

In [ ]:
# Verify installations
import django
import pandas as pd
import matplotlib

print(f"Django version: {django.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

### Project Structure

```
sales_dashboard/
    manage.py
    sales_dashboard/
        __init__.py
        settings.py
        urls.py
        wsgi.py
    dashboard/
        __init__.py
        models.py
        views.py
        urls.py
        services/
            __init__.py
            analytics.py
            charts.py
        templates/
            dashboard/
                index.html
        management/
            commands/
                populate_sales.py
```

---

## 2. Models

The Sale model captures all essential sales transaction data.

### File: `dashboard/models.py`

In [ ]:
# dashboard/models.py

from django.db import models
from django.core.validators import MinValueValidator
from decimal import Decimal


class Sale(models.Model):
    """Represents a sales transaction.
    
    Attributes:
        date: The date of the sale.
        product: Name of the product sold.
        quantity: Number of units sold.
        revenue: Total revenue from the sale.
        region: Geographic region of the sale.
    """
    
    REGION_CHOICES = [
        ('NORTH', 'North'),
        ('SOUTH', 'South'),
        ('EAST', 'East'),
        ('WEST', 'West'),
    ]
    
    date = models.DateField(
        help_text="Date of the sale"
    )
    product = models.CharField(
        max_length=100,
        help_text="Name of the product"
    )
    quantity = models.PositiveIntegerField(
        validators=[MinValueValidator(1)],
        help_text="Number of units sold"
    )
    revenue = models.DecimalField(
        max_digits=10,
        decimal_places=2,
        validators=[MinValueValidator(Decimal('0.01'))],
        help_text="Total revenue in dollars"
    )
    region = models.CharField(
        max_length=10,
        choices=REGION_CHOICES,
        help_text="Geographic region"
    )
    
    class Meta:
        ordering = ['-date']
        indexes = [
            models.Index(fields=['date']),
            models.Index(fields=['product']),
            models.Index(fields=['region']),
        ]
    
    def __str__(self) -> str:
        """Return string representation of the sale."""
        return f"{self.date} - {self.product}: ${self.revenue}"


# Show the model structure
print("Sale Model Fields:")
print("-" * 40)
for field_name in ['date', 'product', 'quantity', 'revenue', 'region']:
    print(f"  - {field_name}")

### Model Design Decisions

1. **REGION_CHOICES**: Using choices ensures data consistency
2. **Validators**: Prevent invalid data at the model level
3. **Indexes**: Optimize queries on frequently filtered fields
4. **DecimalField**: Precise currency handling (never use float for money!)

---

## 3. Data Setup

Populate the database with realistic sample sales data.

### File: `dashboard/management/commands/populate_sales.py`

In [ ]:
# dashboard/management/commands/populate_sales.py

import random
from datetime import date, timedelta
from decimal import Decimal
from typing import List, Tuple

from django.core.management.base import BaseCommand


# Product catalog with base prices
PRODUCTS: List[Tuple[str, Decimal]] = [
    ('Laptop Pro 15', Decimal('1299.99')),
    ('Wireless Mouse', Decimal('49.99')),
    ('USB-C Hub', Decimal('79.99')),
    ('Mechanical Keyboard', Decimal('149.99')),
    ('Monitor 27"', Decimal('399.99')),
    ('Webcam HD', Decimal('89.99')),
    ('Headphones Pro', Decimal('249.99')),
    ('External SSD 1TB', Decimal('129.99')),
]

REGIONS = ['NORTH', 'SOUTH', 'EAST', 'WEST']


class Command(BaseCommand):
    """Management command to populate the database with sample sales data."""
    
    help = 'Populates the database with sample sales data'
    
    def add_arguments(self, parser):
        """Add command arguments."""
        parser.add_argument(
            '--count',
            type=int,
            default=500,
            help='Number of sales records to create (default: 500)'
        )
        parser.add_argument(
            '--days',
            type=int,
            default=365,
            help='Number of days to spread sales over (default: 365)'
        )
    
    def handle(self, *args, **options):
        """Execute the command."""
        from dashboard.models import Sale
        
        count = options['count']
        days = options['days']
        
        # Clear existing data
        Sale.objects.all().delete()
        self.stdout.write('Cleared existing sales data.')
        
        # Generate sales
        sales = []
        today = date.today()
        
        for _ in range(count):
            product_name, base_price = random.choice(PRODUCTS)
            quantity = random.randint(1, 10)
            # Add some price variation (+/- 10%)
            price_variation = Decimal(str(random.uniform(0.9, 1.1)))
            revenue = (base_price * quantity * price_variation).quantize(
                Decimal('0.01')
            )
            
            sale = Sale(
                date=today - timedelta(days=random.randint(0, days)),
                product=product_name,
                quantity=quantity,
                revenue=revenue,
                region=random.choice(REGIONS),
            )
            sales.append(sale)
        
        # Bulk create for efficiency
        Sale.objects.bulk_create(sales)
        
        self.stdout.write(
            self.style.SUCCESS(f'Successfully created {count} sales records.')
        )


# Demonstration of what the data looks like
print("Sample Products:")
for name, price in PRODUCTS[:4]:
    print(f"  {name}: ${price}")
print(f"\nRegions: {REGIONS}")

In [ ]:
# Alternative: Direct data population function (for use in views or shell)

def populate_sales_data(count: int = 500, days: int = 365) -> int:
    """Populate the database with sample sales data.
    
    Args:
        count: Number of sales records to create.
        days: Number of days to spread sales over.
    
    Returns:
        Number of records created.
    """
    import random
    from datetime import date, timedelta
    from decimal import Decimal
    from dashboard.models import Sale
    
    PRODUCTS = [
        ('Laptop Pro 15', Decimal('1299.99')),
        ('Wireless Mouse', Decimal('49.99')),
        ('USB-C Hub', Decimal('79.99')),
        ('Mechanical Keyboard', Decimal('149.99')),
        ('Monitor 27"', Decimal('399.99')),
        ('Webcam HD', Decimal('89.99')),
        ('Headphones Pro', Decimal('249.99')),
        ('External SSD 1TB', Decimal('129.99')),
    ]
    REGIONS = ['NORTH', 'SOUTH', 'EAST', 'WEST']
    
    Sale.objects.all().delete()
    
    sales = []
    today = date.today()
    
    for _ in range(count):
        product_name, base_price = random.choice(PRODUCTS)
        quantity = random.randint(1, 10)
        price_variation = Decimal(str(random.uniform(0.9, 1.1)))
        revenue = (base_price * quantity * price_variation).quantize(
            Decimal('0.01')
        )
        
        sales.append(Sale(
            date=today - timedelta(days=random.randint(0, days)),
            product=product_name,
            quantity=quantity,
            revenue=revenue,
            region=random.choice(REGIONS),
        ))
    
    Sale.objects.bulk_create(sales)
    return count

print("Usage: python manage.py populate_sales --count 500 --days 365")

---

## 4. Analytics Service

A Python class that encapsulates all analytics logic using Pandas.

### File: `dashboard/services/analytics.py`

In [ ]:
# dashboard/services/analytics.py

from datetime import date
from decimal import Decimal
from typing import Dict, List, Optional, Any

import pandas as pd
from django.db.models import QuerySet


class SalesAnalytics:
    """Analytics service for sales data.
    
    Provides methods for calculating various sales metrics and aggregations
    using Pandas for efficient data processing.
    
    Attributes:
        df: Pandas DataFrame containing sales data.
    """
    
    def __init__(self, queryset: QuerySet) -> None:
        """Initialize analytics with a Django queryset.
        
        Args:
            queryset: Django QuerySet of Sale objects.
        """
        # Convert queryset to DataFrame
        self.df = pd.DataFrame(
            list(queryset.values('date', 'product', 'quantity', 'revenue', 'region'))
        )
        
        if not self.df.empty:
            # Convert types for proper analysis
            self.df['date'] = pd.to_datetime(self.df['date'])
            self.df['revenue'] = self.df['revenue'].astype(float)
            self.df['month'] = self.df['date'].dt.to_period('M')
    
    def total_revenue(self) -> float:
        """Calculate total revenue across all sales.
        
        Returns:
            Total revenue as a float.
        """
        if self.df.empty:
            return 0.0
        return float(self.df['revenue'].sum())
    
    def total_quantity(self) -> int:
        """Calculate total quantity of items sold.
        
        Returns:
            Total quantity as an integer.
        """
        if self.df.empty:
            return 0
        return int(self.df['quantity'].sum())
    
    def transaction_count(self) -> int:
        """Get the number of sales transactions.
        
        Returns:
            Number of transactions.
        """
        return len(self.df)
    
    def average_order_value(self) -> float:
        """Calculate average revenue per transaction.
        
        Returns:
            Average order value.
        """
        if self.df.empty:
            return 0.0
        return float(self.df['revenue'].mean())
    
    def sales_by_region(self) -> Dict[str, float]:
        """Calculate total sales by region.
        
        Returns:
            Dictionary mapping region names to total revenue.
        """
        if self.df.empty:
            return {}
        
        by_region = self.df.groupby('region')['revenue'].sum()
        return by_region.to_dict()
    
    def monthly_trend(self) -> pd.DataFrame:
        """Calculate monthly sales trends.
        
        Returns:
            DataFrame with columns: month, revenue, quantity, transactions.
        """
        if self.df.empty:
            return pd.DataFrame(columns=['month', 'revenue', 'quantity', 'transactions'])
        
        monthly = self.df.groupby('month').agg({
            'revenue': 'sum',
            'quantity': 'sum',
            'product': 'count'  # Count transactions
        }).rename(columns={'product': 'transactions'})
        
        monthly = monthly.reset_index()
        monthly['month'] = monthly['month'].astype(str)
        return monthly.sort_values('month')
    
    def top_products(self, n: int = 5) -> pd.DataFrame:
        """Get top-selling products by revenue.
        
        Args:
            n: Number of top products to return.
        
        Returns:
            DataFrame with columns: product, revenue, quantity.
        """
        if self.df.empty:
            return pd.DataFrame(columns=['product', 'revenue', 'quantity'])
        
        by_product = self.df.groupby('product').agg({
            'revenue': 'sum',
            'quantity': 'sum'
        }).reset_index()
        
        return by_product.nlargest(n, 'revenue')
    
    def get_summary(self) -> Dict[str, Any]:
        """Get a complete summary of all analytics.
        
        Returns:
            Dictionary containing all analytics results.
        """
        return {
            'total_revenue': self.total_revenue(),
            'total_quantity': self.total_quantity(),
            'transaction_count': self.transaction_count(),
            'average_order_value': self.average_order_value(),
            'sales_by_region': self.sales_by_region(),
            'monthly_trend': self.monthly_trend(),
            'top_products': self.top_products(),
        }


# Demonstration with sample data
print("SalesAnalytics Methods:")
print("-" * 40)
methods = [
    'total_revenue()',
    'total_quantity()',
    'transaction_count()',
    'average_order_value()',
    'sales_by_region()',
    'monthly_trend()',
    'top_products(n=5)',
    'get_summary()',
]
for method in methods:
    print(f"  - {method}")

In [ ]:
# Demonstration: Creating sample data and running analytics

import pandas as pd
import random
from datetime import date, timedelta

# Create sample DataFrame (simulating Django ORM output)
sample_data = []
products = ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones']
regions = ['NORTH', 'SOUTH', 'EAST', 'WEST']

for i in range(100):
    sample_data.append({
        'date': date.today() - timedelta(days=random.randint(0, 180)),
        'product': random.choice(products),
        'quantity': random.randint(1, 10),
        'revenue': round(random.uniform(50, 500), 2),
        'region': random.choice(regions),
    })

# Create a mock queryset-like object
class MockQuerySet:
    def __init__(self, data):
        self._data = data
    
    def values(self, *fields):
        return [{k: v for k, v in d.items() if k in fields} for d in self._data]

mock_qs = MockQuerySet(sample_data)

# Initialize analytics (we'll simulate the class here)
df = pd.DataFrame(list(mock_qs.values('date', 'product', 'quantity', 'revenue', 'region')))
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

print("Sample Analytics Results:")
print("=" * 40)
print(f"Total Revenue: ${df['revenue'].sum():,.2f}")
print(f"Total Quantity: {df['quantity'].sum():,}")
print(f"Transactions: {len(df)}")
print(f"Average Order: ${df['revenue'].mean():,.2f}")
print("\nSales by Region:")
for region, revenue in df.groupby('region')['revenue'].sum().items():
    print(f"  {region}: ${revenue:,.2f}")

---

## 5. Chart Generation

Matplotlib functions to create charts and return them as base64-encoded images.

### File: `dashboard/services/charts.py`

In [ ]:
# dashboard/services/charts.py

import base64
from io import BytesIO
from typing import Dict, List, Tuple

import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for server use
import matplotlib.pyplot as plt
import pandas as pd


# Color palette for consistent styling
COLORS = {
    'primary': '#3498db',
    'secondary': '#2ecc71',
    'accent': '#e74c3c',
    'neutral': '#95a5a6',
    'regions': ['#3498db', '#2ecc71', '#e74c3c', '#f39c12'],
}


def fig_to_base64(fig: plt.Figure) -> str:
    """Convert a matplotlib figure to a base64-encoded string.
    
    Args:
        fig: Matplotlib figure object.
    
    Returns:
        Base64-encoded PNG image string.
    """
    buffer = BytesIO()
    fig.savefig(buffer, format='png', dpi=100, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    plt.close(fig)
    return image_base64


def create_region_pie_chart(sales_by_region: Dict[str, float]) -> str:
    """Create a pie chart showing sales distribution by region.
    
    Args:
        sales_by_region: Dictionary mapping region names to revenue.
    
    Returns:
        Base64-encoded PNG image string.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    regions = list(sales_by_region.keys())
    values = list(sales_by_region.values())
    
    wedges, texts, autotexts = ax.pie(
        values,
        labels=regions,
        autopct='%1.1f%%',
        colors=COLORS['regions'][:len(regions)],
        explode=[0.02] * len(regions),
        shadow=True,
        startangle=90,
    )
    
    # Style the text
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    ax.set_title('Sales by Region', fontsize=14, fontweight='bold', pad=20)
    
    return fig_to_base64(fig)


def create_monthly_trend_chart(monthly_data: pd.DataFrame) -> str:
    """Create a line chart showing monthly sales trends.
    
    Args:
        monthly_data: DataFrame with 'month' and 'revenue' columns.
    
    Returns:
        Base64-encoded PNG image string.
    """
    fig, ax = plt.subplots(figsize=(10, 5))
    
    months = monthly_data['month'].tolist()
    revenues = monthly_data['revenue'].tolist()
    
    ax.plot(months, revenues, marker='o', linewidth=2, markersize=8,
            color=COLORS['primary'], label='Revenue')
    
    # Fill area under the line
    ax.fill_between(months, revenues, alpha=0.3, color=COLORS['primary'])
    
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('Revenue ($)', fontsize=12)
    ax.set_title('Monthly Sales Trend', fontsize=14, fontweight='bold')
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')
    
    # Format y-axis as currency
    ax.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, p: f'${x:,.0f}')
    )
    
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    return fig_to_base64(fig)


def create_top_products_chart(top_products: pd.DataFrame) -> str:
    """Create a horizontal bar chart showing top-selling products.
    
    Args:
        top_products: DataFrame with 'product' and 'revenue' columns.
    
    Returns:
        Base64-encoded PNG image string.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    products = top_products['product'].tolist()
    revenues = top_products['revenue'].tolist()
    
    # Reverse order so highest is at top
    products = products[::-1]
    revenues = revenues[::-1]
    
    bars = ax.barh(products, revenues, color=COLORS['secondary'], edgecolor='white')
    
    # Add value labels on bars
    for bar, revenue in zip(bars, revenues):
        ax.text(bar.get_width() + max(revenues) * 0.01, bar.get_y() + bar.get_height()/2,
                f'${revenue:,.0f}', va='center', fontsize=10)
    
    ax.set_xlabel('Revenue ($)', fontsize=12)
    ax.set_title('Top Products by Revenue', fontsize=14, fontweight='bold')
    
    # Format x-axis as currency
    ax.xaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, p: f'${x:,.0f}')
    )
    
    # Add some padding on the right for labels
    ax.set_xlim(0, max(revenues) * 1.15)
    
    ax.grid(True, axis='x', alpha=0.3)
    
    plt.tight_layout()
    return fig_to_base64(fig)


print("Chart Functions:")
print("-" * 40)
print("  - fig_to_base64(fig) -> str")
print("  - create_region_pie_chart(sales_by_region) -> str")
print("  - create_monthly_trend_chart(monthly_data) -> str")
print("  - create_top_products_chart(top_products) -> str")

In [ ]:
# Demonstration: Generate sample charts

import matplotlib.pyplot as plt
import pandas as pd
from io import BytesIO
import base64
from IPython.display import HTML, display

# Sample data
region_data = {
    'NORTH': 45000,
    'SOUTH': 32000,
    'EAST': 28000,
    'WEST': 38000,
}

monthly_data = pd.DataFrame({
    'month': ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06'],
    'revenue': [15000, 18500, 22000, 19500, 25000, 28000],
})

product_data = pd.DataFrame({
    'product': ['Laptop Pro 15', 'Monitor 27"', 'Headphones Pro', 'External SSD', 'Keyboard'],
    'revenue': [52000, 38000, 24000, 18000, 12000],
})

# Create region pie chart
fig1, ax1 = plt.subplots(figsize=(6, 4))
ax1.pie(region_data.values(), labels=region_data.keys(), autopct='%1.1f%%',
        colors=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax1.set_title('Sales by Region')
plt.show()

# Create monthly trend chart
fig2, ax2 = plt.subplots(figsize=(8, 4))
ax2.plot(monthly_data['month'], monthly_data['revenue'], marker='o', color='#3498db')
ax2.fill_between(monthly_data['month'], monthly_data['revenue'], alpha=0.3)
ax2.set_title('Monthly Sales Trend')
ax2.set_xlabel('Month')
ax2.set_ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create top products chart
fig3, ax3 = plt.subplots(figsize=(8, 4))
ax3.barh(product_data['product'], product_data['revenue'], color='#2ecc71')
ax3.set_title('Top Products by Revenue')
ax3.set_xlabel('Revenue ($)')
plt.tight_layout()
plt.show()

---

## 6. Views

Django views that tie together the analytics service and chart generation.

### File: `dashboard/views.py`

In [ ]:
# dashboard/views.py

from datetime import date, timedelta
from typing import Any, Dict

from django.http import HttpRequest, HttpResponse
from django.shortcuts import render
from django.views import View

from .models import Sale
from .services.analytics import SalesAnalytics
from .services.charts import (
    create_region_pie_chart,
    create_monthly_trend_chart,
    create_top_products_chart,
)


class DashboardView(View):
    """Main dashboard view displaying sales analytics and charts.
    
    Combines data from the database with analytics processing and
    chart generation to render a complete dashboard.
    """
    
    template_name = 'dashboard/index.html'
    
    def get(self, request: HttpRequest) -> HttpResponse:
        """Handle GET requests to the dashboard.
        
        Args:
            request: The HTTP request object.
        
        Returns:
            Rendered dashboard template.
        """
        # Get date range from query parameters (optional filtering)
        days = int(request.GET.get('days', 365))
        start_date = date.today() - timedelta(days=days)
        
        # Query sales data
        queryset = Sale.objects.filter(date__gte=start_date)
        
        # Run analytics
        analytics = SalesAnalytics(queryset)
        
        # Generate charts
        charts = self._generate_charts(analytics)
        
        # Build context
        context = self._build_context(analytics, charts, days)
        
        return render(request, self.template_name, context)
    
    def _generate_charts(self, analytics: SalesAnalytics) -> Dict[str, str]:
        """Generate all dashboard charts.
        
        Args:
            analytics: SalesAnalytics instance with loaded data.
        
        Returns:
            Dictionary of chart names to base64-encoded images.
        """
        return {
            'region_chart': create_region_pie_chart(analytics.sales_by_region()),
            'trend_chart': create_monthly_trend_chart(analytics.monthly_trend()),
            'products_chart': create_top_products_chart(analytics.top_products()),
        }
    
    def _build_context(self, analytics: SalesAnalytics, 
                       charts: Dict[str, str], days: int) -> Dict[str, Any]:
        """Build the template context.
        
        Args:
            analytics: SalesAnalytics instance.
            charts: Dictionary of generated charts.
            days: Number of days in the date range.
        
        Returns:
            Context dictionary for template rendering.
        """
        return {
            # Key metrics
            'total_revenue': analytics.total_revenue(),
            'total_quantity': analytics.total_quantity(),
            'transaction_count': analytics.transaction_count(),
            'average_order_value': analytics.average_order_value(),
            
            # Charts (base64 encoded)
            'region_chart': charts['region_chart'],
            'trend_chart': charts['trend_chart'],
            'products_chart': charts['products_chart'],
            
            # Data tables
            'sales_by_region': analytics.sales_by_region(),
            'top_products': analytics.top_products().to_dict('records'),
            
            # Filter info
            'days': days,
            'date_range_options': [30, 90, 180, 365],
        }


# Alternative: Function-based view for simpler use cases
def dashboard_view(request: HttpRequest) -> HttpResponse:
    """Function-based dashboard view.
    
    Alternative to class-based view for simpler implementations.
    
    Args:
        request: The HTTP request object.
    
    Returns:
        Rendered dashboard template.
    """
    days = int(request.GET.get('days', 365))
    start_date = date.today() - timedelta(days=days)
    
    queryset = Sale.objects.filter(date__gte=start_date)
    analytics = SalesAnalytics(queryset)
    
    context = {
        'total_revenue': analytics.total_revenue(),
        'total_quantity': analytics.total_quantity(),
        'transaction_count': analytics.transaction_count(),
        'average_order_value': analytics.average_order_value(),
        'region_chart': create_region_pie_chart(analytics.sales_by_region()),
        'trend_chart': create_monthly_trend_chart(analytics.monthly_trend()),
        'products_chart': create_top_products_chart(analytics.top_products()),
        'days': days,
    }
    
    return render(request, 'dashboard/index.html', context)


print("View Components:")
print("-" * 40)
print("  - DashboardView (class-based)")
print("  - dashboard_view (function-based)")

---

## 7. Templates

HTML template for displaying the dashboard with embedded charts.

### File: `dashboard/templates/dashboard/index.html`

In [ ]:
# dashboard/templates/dashboard/index.html

DASHBOARD_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sales Dashboard</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, sans-serif;
            background-color: #f5f6fa;
            color: #2c3e50;
            line-height: 1.6;
        }
        
        .container {
            max-width: 1400px;
            margin: 0 auto;
            padding: 20px;
        }
        
        header {
            background: linear-gradient(135deg, #3498db, #2c3e50);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 30px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        
        header h1 {
            font-size: 2em;
        }
        
        .date-filter {
            display: flex;
            gap: 10px;
        }
        
        .date-filter a {
            color: white;
            text-decoration: none;
            padding: 8px 16px;
            border-radius: 5px;
            background: rgba(255,255,255,0.2);
            transition: background 0.3s;
        }
        
        .date-filter a:hover,
        .date-filter a.active {
            background: rgba(255,255,255,0.4);
        }
        
        .metrics-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }
        
        .metric-card {
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            text-align: center;
        }
        
        .metric-card h3 {
            color: #7f8c8d;
            font-size: 0.9em;
            text-transform: uppercase;
            margin-bottom: 10px;
        }
        
        .metric-card .value {
            font-size: 2.5em;
            font-weight: bold;
            color: #2c3e50;
        }
        
        .metric-card.revenue .value {
            color: #27ae60;
        }
        
        .charts-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(400px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }
        
        .chart-card {
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        
        .chart-card h3 {
            margin-bottom: 15px;
            color: #2c3e50;
        }
        
        .chart-card img {
            width: 100%;
            height: auto;
        }
        
        .full-width {
            grid-column: 1 / -1;
        }
        
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
        }
        
        th, td {
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #ecf0f1;
        }
        
        th {
            background: #f8f9fa;
            font-weight: 600;
        }
        
        tr:hover {
            background: #f8f9fa;
        }
        
        footer {
            text-align: center;
            padding: 20px;
            color: #7f8c8d;
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <h1>Sales Dashboard</h1>
            <div class="date-filter">
                {% for option in date_range_options %}
                <a href="?days={{ option }}" 
                   {% if days == option %}class="active"{% endif %}>
                    {{ option }} Days
                </a>
                {% endfor %}
            </div>
        </header>
        
        <!-- Key Metrics -->
        <div class="metrics-grid">
            <div class="metric-card revenue">
                <h3>Total Revenue</h3>
                <div class="value">${{ total_revenue|floatformat:2 }}</div>
            </div>
            <div class="metric-card">
                <h3>Units Sold</h3>
                <div class="value">{{ total_quantity }}</div>
            </div>
            <div class="metric-card">
                <h3>Transactions</h3>
                <div class="value">{{ transaction_count }}</div>
            </div>
            <div class="metric-card">
                <h3>Avg. Order Value</h3>
                <div class="value">${{ average_order_value|floatformat:2 }}</div>
            </div>
        </div>
        
        <!-- Charts -->
        <div class="charts-grid">
            <div class="chart-card">
                <h3>Sales by Region</h3>
                <img src="data:image/png;base64,{{ region_chart }}" alt="Region Chart">
            </div>
            <div class="chart-card">
                <h3>Top Products</h3>
                <img src="data:image/png;base64,{{ products_chart }}" alt="Products Chart">
            </div>
            <div class="chart-card full-width">
                <h3>Monthly Trend</h3>
                <img src="data:image/png;base64,{{ trend_chart }}" alt="Trend Chart">
            </div>
        </div>
        
        <!-- Data Tables -->
        <div class="charts-grid">
            <div class="chart-card">
                <h3>Regional Breakdown</h3>
                <table>
                    <thead>
                        <tr>
                            <th>Region</th>
                            <th>Revenue</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for region, revenue in sales_by_region.items %}
                        <tr>
                            <td>{{ region }}</td>
                            <td>${{ revenue|floatformat:2 }}</td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
            <div class="chart-card">
                <h3>Top Products Details</h3>
                <table>
                    <thead>
                        <tr>
                            <th>Product</th>
                            <th>Revenue</th>
                            <th>Qty</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for product in top_products %}
                        <tr>
                            <td>{{ product.product }}</td>
                            <td>${{ product.revenue|floatformat:2 }}</td>
                            <td>{{ product.quantity }}</td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
        </div>
        
        <footer>
            <p>Sales Dashboard | Data from last {{ days }} days</p>
        </footer>
    </div>
</body>
</html>
'''

print("Template Features:")
print("-" * 40)
print("  - Responsive grid layout")
print("  - Metric cards with styling")
print("  - Embedded base64 chart images")
print("  - Data tables for details")
print("  - Date range filter navigation")

---

## 8. URLs

URL configuration for the dashboard application.

### File: `dashboard/urls.py`

In [ ]:
# dashboard/urls.py

from django.urls import path
from . import views

app_name = 'dashboard'

urlpatterns = [
    # Main dashboard view
    path('', views.DashboardView.as_view(), name='index'),
    
    # Alternative: function-based view
    # path('', views.dashboard_view, name='index'),
]

print("Dashboard URLs:")
print("-" * 40)
print("  /           -> Dashboard home")
print("  /?days=30   -> Last 30 days")
print("  /?days=90   -> Last 90 days")
print("  /?days=365  -> Last year")

In [ ]:
# sales_dashboard/urls.py (main project URLs)

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('dashboard.urls')),
]

print("Project URL Configuration:")
print("-" * 40)
print("  /admin/  -> Django admin")
print("  /        -> Dashboard app")

---

## 9. Running the App

### Complete Setup Instructions

In [ ]:
# Step-by-step setup instructions

setup_instructions = '''
=== Sales Dashboard Setup Guide ===

1. CREATE PROJECT STRUCTURE
   --------------------------
   # Create Django project
   django-admin startproject sales_dashboard
   cd sales_dashboard
   
   # Create dashboard app
   python manage.py startapp dashboard
   
   # Create services directory
   mkdir dashboard/services
   touch dashboard/services/__init__.py
   
   # Create templates directory
   mkdir -p dashboard/templates/dashboard
   
   # Create management command directory
   mkdir -p dashboard/management/commands
   touch dashboard/management/__init__.py
   touch dashboard/management/commands/__init__.py

2. CONFIGURE SETTINGS
   -------------------
   # In sales_dashboard/settings.py, add to INSTALLED_APPS:
   INSTALLED_APPS = [
       ...
       'dashboard',
   ]

3. ADD CODE FILES
   ---------------
   Copy the code from this notebook into:
   - dashboard/models.py
   - dashboard/services/analytics.py
   - dashboard/services/charts.py
   - dashboard/views.py
   - dashboard/urls.py
   - dashboard/templates/dashboard/index.html
   - dashboard/management/commands/populate_sales.py
   - sales_dashboard/urls.py

4. RUN MIGRATIONS
   ---------------
   python manage.py makemigrations dashboard
   python manage.py migrate

5. POPULATE SAMPLE DATA
   ---------------------
   python manage.py populate_sales --count 500 --days 365

6. START SERVER
   -------------
   python manage.py runserver

7. VIEW DASHBOARD
   ---------------
   Open browser to: http://127.0.0.1:8000/
   
   Try different date filters:
   - http://127.0.0.1:8000/?days=30
   - http://127.0.0.1:8000/?days=90
   - http://127.0.0.1:8000/?days=365
'''

print(setup_instructions)

In [ ]:
# Quick start script (all-in-one)

quick_start_script = '''
#!/bin/bash
# Quick start script for Sales Dashboard

# Create and activate virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\\Scripts\\activate

# Install dependencies
pip install django pandas matplotlib

# Create project
django-admin startproject sales_dashboard
cd sales_dashboard
python manage.py startapp dashboard

# Create directory structure
mkdir -p dashboard/services
mkdir -p dashboard/templates/dashboard
mkdir -p dashboard/management/commands

# Create empty __init__.py files
touch dashboard/services/__init__.py
touch dashboard/management/__init__.py
touch dashboard/management/commands/__init__.py

echo "Project structure created!"
echo "Now add the code files and run:"
echo "  python manage.py makemigrations dashboard"
echo "  python manage.py migrate"
echo "  python manage.py populate_sales"
echo "  python manage.py runserver"
'''

print("Quick Start Script:")
print("=" * 50)
print(quick_start_script)

In [ ]:
# Complete settings.py configuration

settings_config = '''
# Add these to sales_dashboard/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'dashboard',  # Add this line
]

# Optional: Configure logging for debugging
LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
        },
    },
    'root': {
        'handlers': ['console'],
        'level': 'INFO',
    },
}
'''

print("Settings Configuration:")
print("=" * 50)
print(settings_config)

---

## 10. Challenges

### Extension Ideas

Take this dashboard further with these challenges:

In [ ]:
# Challenge 1: Date Range Filters
# Add custom date range selection with start/end date pickers

challenge_1 = '''
CHALLENGE 1: Custom Date Range Filters
======================================

Goal: Add a date picker to filter sales by custom date range.

Implementation Steps:
1. Add start_date and end_date query parameters to the view
2. Update the template with date input fields
3. Add JavaScript for date picker functionality
4. Validate date inputs in the view

Code Hint:
----------
from datetime import datetime

def get(self, request):
    start_date_str = request.GET.get('start_date')
    end_date_str = request.GET.get('end_date')
    
    if start_date_str and end_date_str:
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()
        queryset = Sale.objects.filter(date__range=[start_date, end_date])
    else:
        queryset = Sale.objects.all()
'''

print(challenge_1)

In [ ]:
# Challenge 2: Export to CSV/Excel
# Add data export functionality

challenge_2 = '''
CHALLENGE 2: Export Data to CSV/Excel
=====================================

Goal: Add buttons to export dashboard data as CSV or Excel files.

Implementation Steps:
1. Create an export view that returns CSV/Excel response
2. Use Pandas to_csv() or to_excel() with BytesIO
3. Set appropriate HTTP headers for file download
4. Add export buttons to the template

Code Hint:
----------
from django.http import HttpResponse
import pandas as pd
from io import BytesIO

def export_csv(request):
    """Export sales data as CSV."""
    queryset = Sale.objects.all()
    df = pd.DataFrame(list(queryset.values()))
    
    response = HttpResponse(content_type='text/csv')
    response['Content-Disposition'] = 'attachment; filename="sales_export.csv"'
    df.to_csv(response, index=False)
    return response

def export_excel(request):
    """Export sales data as Excel."""
    queryset = Sale.objects.all()
    df = pd.DataFrame(list(queryset.values()))
    
    buffer = BytesIO()
    df.to_excel(buffer, index=False, sheet_name='Sales')
    buffer.seek(0)
    
    response = HttpResponse(
        buffer.getvalue(),
        content_type='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    )
    response['Content-Disposition'] = 'attachment; filename="sales_export.xlsx"'
    return response
'''

print(challenge_2)

In [ ]:
# Challenge 3: REST API
# Create a REST API for the analytics

challenge_3 = '''
CHALLENGE 3: REST API for Analytics
====================================

Goal: Create REST endpoints that return JSON analytics data.

Implementation Steps:
1. Install Django REST Framework: pip install djangorestframework
2. Create serializers for the analytics data
3. Build API views using DRF ViewSets or APIViews
4. Add URL routing for API endpoints

Code Hint:
----------
from django.http import JsonResponse
from django.views import View

class AnalyticsAPIView(View):
    """API endpoint for sales analytics."""
    
    def get(self, request):
        days = int(request.GET.get('days', 365))
        start_date = date.today() - timedelta(days=days)
        
        queryset = Sale.objects.filter(date__gte=start_date)
        analytics = SalesAnalytics(queryset)
        
        return JsonResponse({
            'total_revenue': analytics.total_revenue(),
            'total_quantity': analytics.total_quantity(),
            'transaction_count': analytics.transaction_count(),
            'average_order_value': analytics.average_order_value(),
            'sales_by_region': analytics.sales_by_region(),
            'monthly_trend': analytics.monthly_trend().to_dict('records'),
            'top_products': analytics.top_products().to_dict('records'),
        })

# URLs:
# path('api/analytics/', AnalyticsAPIView.as_view(), name='api-analytics'),
'''

print(challenge_3)

In [ ]:
# Challenge 4: Real-time Updates with HTMX
# Add real-time dashboard updates without full page reloads

challenge_4 = '''
CHALLENGE 4: Real-time Updates with HTMX
=========================================

Goal: Update dashboard metrics without full page reloads using HTMX.

Implementation Steps:
1. Include HTMX library in your template
2. Create partial template for metrics section
3. Add HTMX attributes for auto-refresh
4. Create endpoint that returns partial HTML

Code Hint:
----------
<!-- In base template, add HTMX -->
<script src="https://unpkg.com/htmx.org@1.9.10"></script>

<!-- Metrics section with auto-refresh -->
<div id="metrics" 
     hx-get="/dashboard/metrics/"
     hx-trigger="every 30s"
     hx-swap="innerHTML">
    {% include "dashboard/_metrics.html" %}
</div>

# View for partial update
def metrics_partial(request):
    """Return just the metrics HTML fragment."""
    analytics = SalesAnalytics(Sale.objects.all())
    context = {
        'total_revenue': analytics.total_revenue(),
        'total_quantity': analytics.total_quantity(),
        'transaction_count': analytics.transaction_count(),
        'average_order_value': analytics.average_order_value(),
    }
    return render(request, 'dashboard/_metrics.html', context)
'''

print(challenge_4)

In [ ]:
# Challenge 5: Interactive Charts with Plotly
# Replace static Matplotlib charts with interactive Plotly charts

challenge_5 = '''
CHALLENGE 5: Interactive Charts with Plotly
============================================

Goal: Replace static Matplotlib charts with interactive Plotly charts.

Implementation Steps:
1. Install Plotly: pip install plotly
2. Create Plotly chart functions
3. Return chart HTML instead of base64 images
4. Update template to use |safe filter for HTML

Code Hint:
----------
import plotly.graph_objects as go
import plotly.express as px

def create_interactive_trend_chart(monthly_data):
    """Create interactive line chart with Plotly."""
    fig = px.line(
        monthly_data,
        x='month',
        y='revenue',
        title='Monthly Sales Trend',
        markers=True
    )
    
    fig.update_layout(
        hovermode='x unified',
        showlegend=False
    )
    
    # Return HTML div that can be embedded in template
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

# In template:
# {{ trend_chart|safe }}
'''

print(challenge_5)

---

## Summary

### What We Built

A complete fullstack Sales Dashboard that demonstrates:

1. **Django ORM** - Database model definition and queries
2. **SQL Integration** - Using Django's ORM for efficient database operations
3. **Pandas Analytics** - Data processing and aggregation
4. **Matplotlib Visualization** - Chart generation as embeddable images
5. **Web Development** - Views, templates, and URL routing

### Architecture Patterns Used

- **Service Layer**: Analytics logic separated from views
- **Class-based Views**: Organized, reusable view logic
- **Template Inheritance**: Could extend for more pages
- **Query Optimization**: Using `values()` to minimize database overhead

### Key Takeaways

1. **Separation of Concerns**: Keep analytics, charts, and views separate
2. **Type Hints**: Document expected types for better code quality
3. **Base64 Images**: Embed charts directly in HTML without file storage
4. **Pandas + Django**: Efficient bridge between ORM and data analysis
5. **Progressive Enhancement**: Start simple, add features incrementally

In [ ]:
# Final summary of all components

print("="*60)
print("FULLSTACK DATA APPLICATION: SALES DASHBOARD")
print("="*60)
print()
print("Components Created:")
print("-"*40)
print("  1. Sale Model (dashboard/models.py)")
print("  2. Data Population Command (management/commands/)")
print("  3. SalesAnalytics Service (services/analytics.py)")
print("  4. Chart Generation (services/charts.py)")
print("  5. DashboardView (views.py)")
print("  6. Dashboard Template (templates/dashboard/index.html)")
print("  7. URL Configuration (urls.py)")
print()
print("Technologies Integrated:")
print("-"*40)
print("  - Django 4.x (Web Framework)")
print("  - SQLite/PostgreSQL (Database)")
print("  - Pandas (Data Analysis)")
print("  - Matplotlib (Visualization)")
print()
print("Features:")
print("-"*40)
print("  - Key metrics display (revenue, quantity, transactions)")
print("  - Regional sales breakdown (pie chart)")
print("  - Monthly trend analysis (line chart)")
print("  - Top products ranking (bar chart)")
print("  - Date range filtering (30/90/180/365 days)")
print()
print("Extension Challenges:")
print("-"*40)
print("  1. Custom date range pickers")
print("  2. CSV/Excel export")
print("  3. REST API endpoints")
print("  4. Real-time updates with HTMX")
print("  5. Interactive Plotly charts")
print()
print("="*60)